In [ ]:
import math

# r = interest rate
# n = number of compounding periods
# s = volatility
# S = original stock price
# K = strike price

# p = probability of u
# u = upw  move
# d = down move

##############################################################################


class CoxRossRubenstein:

	def choose(self,a,b):
		return math.factorial(a) // math.factorial(b) // math.factorial(a-b)

	def call_payoff(self,S,K):
		return max(S-K, 0.0)

	def put_payoff(self,S,K):
		return max(K - S, 0.0)

	def build_end_node(self,p,i,n,u,d,S,K):

		node  = (p**i) * (1.0-p)**(n-i)
		node *= self.choose(n,i)

		price_move = (u**i) * (d**(n-i))

		call_node = node * self.call_payoff( price_move * S , K )

		put_node  = node * self.put_payoff( price_move * S , K )

		return call_node, put_node

	def __init__(self,r,n,s,T,S,K):

		dT = float(T)/n

		u = math.exp( s * math.sqrt(dT) ) # return of up move CORRECT
		d = 1 / u                         # return of down move CORRECT

		p  = math.exp(r*dT) - d 		  # probability of up move CORRECT
		p /= u - d                        # probability of up move CORRECT

		final_call_nodes,final_put_nodes = [], []

		for i in range(n+1): # x: 0->n

			final_call_node , final_put_node = self.build_end_node(p,i,n, u,d, S,K)

			final_call_nodes.append(final_call_node)

			final_put_nodes.append(final_put_node)

		discount = math.exp(-1 * r * T)

		self.call_price = discount * sum(final_call_nodes)
		self.call_price = round(self.call_price,2)

		self.put_price = discount * sum(final_put_nodes)
		self.put_price = round(self.put_price,2)


	def get_put_price(self):
		return self.put_price

	def get_call_price(self):
		return self.call_price

	def summary(self):

		print('\n\tCOX ROSS RUBENSTEIN')
		print()
		print('European Call Price:\t$' , self.get_call_price())
		print()
		print('European Put Price:\t$' , self.get_put_price())
		print()	

##############################################################################

def normalcdf(x):
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

class BlackScholesMerton:

	def __init__(self,r,s,t,S,K):

		d1 = ( math.log(S/K) + (r + 0.5*s**2)*t ) / ( s * math.sqrt(t) )
		d2 = d1 - s * math.sqrt(t)

		self.call_price = S*normalcdf(d1) - (K*math.exp(-r*t))*normalcdf(d2)

		self.call_price = round(self.call_price,2)

		self.put_price = (K*math.exp(-r*t))*normalcdf(-d2) - S*normalcdf(-d1)

		self.put_price = round(self.put_price,2)

	def get_put_price(self):
		return self.put_price

	def get_call_price(self):
		return self.call_price

	def summary(self):

		print("\n\tBLACK SCHOLES MERTON:")
		print()
		print('European Call Price:\t$' , self.get_call_price())
		print()
		print('European Put Price:\t$' , self.get_put_price())
		print()	

##############################################################################

from numpy import sqrt,cumprod,mean,exp
from numpy import random as rd

class MonteCarlo:

	def call_itm(self,walks,K):
		return ( walks - K ) > 0

	def put_itm(self,walks,K):
		return (K - walks) > 0

	def call_payoff(self,walks,K):
		return walks - K

	def put_payoff(self,walks,K):
		return K - walks

	def __init__(self,r,n,s,t,S,K):

		days = t * 365
		daily_vol = s / sqrt(days)

		daily_walks = cumprod( 1 + rd.randn(n, days) * daily_vol , 1 )

		walks = S * exp(r * t) * daily_walks
		
		walks_ends = walks[:,-1]

		calls = self.call_payoff(walks_ends,K)
		calls *= self.call_itm(walks_ends,K)

		puts = self.put_payoff(walks_ends,K)
		puts *= self.put_itm(walks_ends,K)

		self.call_price = round( mean(calls) ,2)

		self.put_price = round( mean(puts) ,2)

	def get_put_price(self):
		return self.put_price

	def get_call_price(self):
		return self.call_price

	def summary(self):

		print("\n\tMONTE CARLO (NORMAL):")
		print()
		print('European Call Price:\t$' , self.get_call_price())
		print()
		print('European Put Price:\t$' , self.get_put_price())
		print()	

##############################################################################

# class ReinforcementLearning:	

##############################################################################